$ pip install tqdm HaversineAPI

In [ ]:
import sys, os, re, json, getpass
from tqdm.notebook import tqdm
from Haversine.haversine import Waypoints, Routes

hr = Routes()
hr.username = 'eddo888'
hr.password = getpass.getpass()

In [2]:
routes = dict(map(lambda x: (x['name'], x), hr.list()))

In [3]:
names = list(routes.keys())
names

['EGLL-LFPG',
 'YSSY-YMML',
 'NZAA-NZCH',
 'KEDW-KNKX',
 'KEDW-KSFO',
 'LIRN-LMML',
 'KNKX-KEDW',
 'KOPF-KTTS',
 'YDBY-YPPH',
 'LIRA-LFKJ',
 'KLAX-KEDW',
 'YSSY-YJBY',
 'OMAA-OTHH',
 'ZBAD-VHHH',
 'YPDN-YDBY',
 'RJAA-RJOO',
 'LIRI-LICR',
 'KEDW-KLAX',
 'MTCH-MTPP',
 'YSSY-YPAD',
 'PHKO-PHNL',
 'LIRA-LIPZ',
 'YBBN-YBCS',
 'KBOS-CYUL',
 'MPMG-MPMG',
 'LIRA-LIRQ',
 'LFPG-EGLL',
 'KRAL-KFFZ',
 'YBCS-YPDN',
 'PGWT-RJOA',
 'YPPH-YPAD',
 'KLFI-KDCA',
 'KRAL-KSFO',
 'OMAA-YSSY',
 'YMHB-YMML',
 'KRAL-KSEA',
 'KTTS-KPBI',
 'EGPH-EIDW',
 'LIRN-LIRA',
 'RPLL-RJAA',
 'YPAD-YMML',
 'HEPS-HECA',
 'KPSP-KPSP',
 'KSFO-KLAX',
 'YSCB-YSSY',
 'YSCB-YMML',
 'HKMO-HKJK',
 'KLAX-KSFO',
 'HECA-OMDB',
 'KLAX-KSEA',
 'KJFK-KIAD',
 'LPPT-LEPA',
 'VGBR-VQPR',
 'KRAL-KLPC',
 'CYYC-CYEG',
 'KLPC-KRAL',
 'KBFI-KSFO',
 'YBBN-YMML',
 'YBBN-YSSY',
 'YBBN-YAYE',
 'KSEA-KRAL',
 'KIAD-KTTS',
 'KSFO-KRAL',
 'KJFK-KDCA',
 'LEMD-LEBL',
 'YSSY-YBBN',
 'KSFO-KSEA',
 'EGLL-EGPH',
 'YPAD-YMHB',
 'KJFK-KAFJ',
 'BIVM-BIBA',
 'YMML

In [4]:
mapper = dict(
    ADEP='origin',
    DEPRWY='departure_runway',
    SID='sid',
    #SIDTRANS WOL
    ADES='destination',
    DESRWY='arrival_runway',
    STAR='star',
    #STARTRANS ARBEY
    APP='approach',
)

In [5]:
def process(file):
    path = []
    fms = open(file).read()
    route = dict()
    pattern = re.compile('^(\S+)\s+(\S+)$')
    waypoint = re.compile('^(\d+)\s+(\S+)\s*.*$')

    for line in fms.split('\n'):
        #print(line)
        match = pattern.match(line)
        if match:
            #print(match.groups())
            key = match.group(1)
            value = match.group(2)        
            if key in mapper.keys():
                route[mapper[key]] = value
            continue
        match = waypoint.match(line)
        if match:
            #print(match.groups())
            path.append(match.group(2))

    route['name'] = f'{route["origin"]}-{route["destination"]}'
    route['path'] = ' '.join(path[1:-1])
    return route

In [1]:
def upload(route):
    name = route["name"]
    new_file = f'{name}.json'

    if os.path.exists(new_file):
        print(f'={new_file}')
        with open(new_file) as input:
            route = json.load(input)
    else:
        print(f'+{new_file}')
        with open(new_file,'w') as output:
            json.dump(route, output, indent='\t')

    if name in names:
        print(f'-{name}')
        hr.delete(name)
        #return
  
    result = hr.create(new_file)
    if result and result['result'] == 'success':
        print(f'+{result["route"]["name"]}')
    else:
        try:
            print(f'~{name}')
            hr.suggest(route['origin'], route['destination'], first=True, output=new_file)
            hr.create(new_file)
        except:
            pass
        
    return

In [12]:
for file in tqdm(list(filter(lambda x: os.path.isfile(x) and x.endswith('.fms'), os.listdir('.')))):
    try:
        route = process(file)
        #json.dump(route, sys.stdout, indent='\t')
    except:
        continue
    result = upload(route)


  0%|          | 0/1 [00:00<?, ?it/s]

=YSSY-YMML.json
-YSSY-YMML
+YSSY-YMML
+YSSY-YMML
